# Train the model by preparing the dataset

In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
from src import preparedata
from src import modelarea
from src import trainarea

print(tf.__version__)
print(tf.config.list_physical_devices(
    device_type=None
))

params=json.load(open('params/params.json','r'))

2023-01-19 19:34:37.851451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 19:34:38.040513: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-19 19:34:40.329061: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/dahala/miniconda3/envs/dlashok/lib/
2023-01-19 19:34:40.330099: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: lib

2.10.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
#prepare data
dataset=preparedata.readGPDData(params['dataprepinargs'])
dataset.preparedata()

  0%|          | 0/31 [00:00<?, ?it/s]

In [3]:
#prepare model
landslidehazard=modelarea.lhmodel(params['modelparam'])
landslidehazard.preparemodel()

2023-01-19 19:36:04.300825: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-19 19:36:04.946424: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10404 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:b1:00.0, compute capability: 6.1


In [4]:
trainarea.trainmodel(landslidehazard.model,dataset.X_train,dataset.Y_train,params['trainparam'])
landslidehazard.model.save_weights('savedweights/final_model.h5')

Epoch 1/100
222/223 [============================>.] - ETA: 0s - loss: 1.4178 - gpdmetric: 0.4953
Epoch 1: val_loss improved from inf to 1.05094, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
223/223 [==============================] - 28s 89ms/step - loss: 1.4175 - gpdmetric: 0.4956 - val_loss: 1.0509 - val_gpdmetric: 0.5377
Epoch 2/100
221/223 [============================>.] - ETA: 0s - loss: 1.2407 - gpdmetric: 0.6091
Epoch 2: val_loss improved from 1.05094 to 0.97718, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
223/223 [==============================] - 19s 85ms/step - loss: 1.2399 - gpdmetric: 0.6096 - val_loss: 0.9772 - val_gpdmetric: 0.6201
Epoch 3/100
222/223 [============================>.] - ETA: 0s - loss: 1.1536 - gpdmetric: 0.6821
Epoch 3: val_loss improved from 0.97718 to 0.93639, saving model to checkpoints/
INFO:tensorflow:Assets written to: checkpoints/assets
223/223 [==========================

NameError: name 'model' is not defined

In [5]:
landslidehazard.model.save_weights('savedweights/final_model.h5')

# Perform inference by calculating landslide hazrd

In [13]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
from src import preparedata
from src import modelarea
from src import inference
from src import savehazard

print(tf.__version__)
print(tf.config.list_physical_devices(
    device_type=None
))
params=json.load(open('params/params.json','r'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
2.10.1
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [7]:
rp=10
dataset=preparedata.readGPDData(params['dataprepinargs'])
dataset.preparedatainference(rp)

  0%|          | 0/31 [00:00<?, ?it/s]

In [8]:
landslidehazard=modelarea.lhmodel(params['modelparam'])
landslidehazard.preparemodel()
haz=inference.inferenceLH(model=landslidehazard.model,model_weights='savedweights/final_model.h5',xdata=dataset.Xinference,rp=rp)

2219/2219 [==============================] - 8s 3ms/step


In [14]:
savehazard.save_predicted_Haz(ids=dataset.InferenceID,prediction=haz,hazcol='haz_10rp',sufile='Data/SlopeUnits/SlopeUnits_V3.shp',outfile='Data/Results/LsHaz10.shp')